In [ ]:
# default_exp pubmed

# pubmed

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import re
import random
from pymongo import MongoClient
import pandas as pd
import multiprocessing as mp
from tqdm.notebook import tqdm

In [ ]:
URL = 'https://www.ncbi.nlm.nih.gov'

In [ ]:
user_agent = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
        ]

### WebDriver

Start WebDriver

* [Install ChromeDriver](https://chromedriver.chromium.org/)


In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

In [ ]:
browser = webdriver.Chrome(options=options, executable_path='./chromedriver')

In [ ]:
browser.page_source

'<html><head></head><body></body></html>'

### Database

Create MongoDB container from [image](https://hub.docker.com/_/mongo) and expose to `localhost:32768`

In [ ]:
# docker run --name research-mongo -d mongo:latest

In [ ]:
MONGO_HOST = 'mongodb://localhost:32768'

In [ ]:
client = MongoClient(MONGO_HOST)

In [ ]:
# create or load pubmed database
db = client.PubMed

### Utils

In [ ]:
#export
def get_max_pages(keywords):
    # build search link
    url = f'{URL}/pubmed/?term={keywords}'
    browser.get(url)
    browser.implicitly_wait(1) # wait to avoid traffic
    
    s = BeautifulSoup(browser.page_source, 'html.parser')
    max_pages = int(s.find('input', {'id': 'pageno2'}).get('last'))
    return max_pages

In [ ]:
search_keywords = 'electroencephalogram spectroscopy'

In [ ]:
max_pages = get_max_pages(search_keywords); max_pages

197

In [ ]:
# browser.page_source

#### Headers

In [ ]:
#export
def request_headers(url):
    return {
            'User-Agent':random.choice(user_agent),
            'Referer': url,
            'Connection':'keep-alive',
            'Host':'www.ncbi.nlm.nih.gov'}

#### Mongo DB

In [ ]:
#export
def write_db(data):
    # check if already exists
#     if db.pubmed_meta.find({'url': data['url']}).limit(1):
#         print(data['url'], 'already exists')
#     else:
    db.pubmed_meta.insert_one(data)

#### Extractors

In [ ]:
def get_orgs(bs):
    try:
        orgs = (bs
        .find('dl',{'class': 'ui-ncbi-toggler-slave'})
        .find_all('dd'))
        return [o.get_text() for o in orgs]
    except:
        return 'ORGANIZATION_NA'

In [ ]:
def get_kwords(bs):
    try:
        return (bs
           .find('div', {'class': 'keywords'})
           .find('p')
           .get_text()
           .split())
    except:
        return 'KEYWORDS_NA'

In [ ]:
#export
def extract_page(url):
    browser.implicitly_wait(random.randint(2,3))
    html = requests.get(url, headers=request_headers(url))
    bs = BeautifulSoup(html.text, 'html.parser')
    
    title = bs.find('title').get_text()
    authors = bs.find('div', {'class': 'auths'}).get_text()
    orgs = get_orgs(bs)
    kwords = get_kwords(bs)
    cit = bs.find('div', {'class': 'cit'}).get_text()
#     date = re.findall(r'\d{4}\s\w{3}\s\d{2}', citation)[0]
    return dict(title=title, authors=authors, organizations=orgs,
                keywords=kwords, citation=cit, url=url)

In [ ]:
#export
def extract_and_write(url):
    data = extract_page(url)
    return write_db(data)

#### TODO: this shoud be multi threaded not processing

In [ ]:
#export
def process_many(urls):
    p = mp.Pool()
    for u in urls:
        p.apply_async(extract, args=(u,))
    
    p.close()
    p.join()

## Crawler

In [ ]:
#export
def crawl_list(page):
    bs = BeautifulSoup(page, 'html.parser')
    divs = bs.find_all('div', {'class': 'rslt'})
    for d in tqdm(divs):
        # get paper's link
        u = URL + d.find('a').get('href')
        extract_and_write(u)

In [ ]:
#export
def crawl_pubmed(keywords):
    url = f'{URL}/pubmed/?term={search_keywords}'
    browser.get(url)
    max_pages = get_max_pages(keywords)
    for _ in tqdm(range(max_pages)):
        crawl_list(browser.page_source)
        browser.implicitly_wait(1)
        # Click next buttion to navigate to the next page
        browser.find_element_by_xpath('//*[@title="Next page of results"]').click()

In [ ]:
crawl_pubmed(search_keywords)

## TODO:

* crawl for g.tect mentions products in papers